In [1]:
import os
import xlsxwriter
import numpy as np
from pylab import plot, grid, xlmlim, ylim, xlabel, ylabel, linspace, figure, legend
import pandas as pd
from pandas import isnull

In [2]:
def FindDel(S):
    """This function takes a the first string in a list,(S) that starts with ENG, it finds the nomancature 
    the file uses for the begining of each file, e.g ENG44007 or ENGINX00305258, it shall also work if more numbers
    are added or the name changes so long as ENG followed by anything other than "." or "_" or " " is still 
    the start of each file"""
    for i in S:
        if i[0:3] == 'ENG':
            n = 1
            for j in i:
                if j == '.':
                    break
                elif j == ' ':
                    break
                elif j == '_':
                    break
                else: 
                    n += 1 
            return n
        else:
            continue


def InitialRun(S):
    """This takes S a list of filenames and returns the first run index"""
    last_int_index = FindDel(S) -1
    first_int_index = 0 
    for i in S:                                                          
        if i[0:3] == 'ENG':
            for j in range(last_int_index):
                try:
                    int(i[j])
                    run_number = i[first_int_index:last_int_index]
                    break
                except:
                    first_int_index += 1
        else:
            continue
        break
    return((run_number))

def CycleRuns(S):
    """This takes a list of filenames and returns a list of all the runs in the file"""
    Start_run = int(InitialRun(S))
    Last_run = int(InitialRun(reversed(S)))
    Runs = range(Start_run,Last_run)
    return (list(Runs))

def RunIndex(s,n): #ENGINX00305258
    """s is a string in the for m of a filename from the enginx data and returns it's run number, n is the FindDel"""
    last_int_index = n-1
    first_int_index = 0
    for j in range(last_int_index):
        try:
            int(s[j])
            return s[first_int_index:last_int_index]
        except:
            first_int_index += 1

### Making spreadsheet of file names 

In [51]:
directory = "D:\\"
array_names = []
file_years = []
long_array_names = []
for foldername in os.listdir(directory):
    if foldername[0:5] == 'cycle':
        n = FindDel(os.listdir("D:\\"+foldername))
        print(n)
        for filename in os.listdir("D:\\"+foldername):
            if filename[n:] in array_names: #stop repeated file names from entering 
                pass
            elif len(filename) < n+1: # Stop useless file names from entering (e.g copy)
                pass
            elif filename[n] in ("S", "s", "N", "n"):# Stop temporary files from entering (e.g .s01 or .n001)
                try:
                    int(filename[n+1]) # all temp files are 's' or 'n' followed by a int
                    temp_files.append(filename)
                except ValueError: 
                    array_names.append(filename[n:])
                    long_array_names.append(filename)
                    file_years.append(foldername)
            else: # most unique file names go here 
                array_names.append(filename[n:])
                long_array_names.append(filename)
                file_years.append(foldername)
                
#print(file_years)
#print(array_names)
#print(long_array_names)

workbook   = xlsxwriter.Workbook('Filenames.xlsx')

worksheet1 = workbook.add_worksheet()
worksheet2 = workbook.add_worksheet()

worksheet1.write_column('A1', array_names)
worksheet2.write_column('A1', long_array_names)
worksheet1.write_column('B1', file_years)
worksheet2.write_column('B1', file_years)
workbook.close()

9
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
['NG45688.RAW', 'NG45995_Strain.txt', 'RAW', 'Current.txt', 'dash_controls.txt', 'MonCounts.txt', 'Status.txt', 'Theta.txt', 'X.txt', 'Y.txt', 'Z.txt', 'Slit_H.txt', 'Slit_V.txt', 'Slit_X.txt', 'Temp4.txt', 'Load.txt', 'Pos.txt', 'Strain.txt', 'Temperature.txt', 'Position.txt', 'RUN.txt', '_copy.log', 'to cycle_04_1.lnk', 'nxs', 'raw', 'cyba_a.txt', 'cyba_b.txt', 'cyba_c.txt', 'ICPdebug.txt', 'ICPevent.txt', 'slitpos.txt', 'w.txt', 'x.txt', 'y.txt', 'z.txt', 'Jaws1_H.txt', 'Jaws1_V.txt', 'Jaws2_H.txt', 'Jaws2_V.txt', 'C6_MASTER_FREQUENCY.txt', 'C6_MASTER_PHASE.txt', 'C6_SLAVE_FREQUENCY.txt', 'C6_SLAVE_PHASE.txt', 'C9_MASTER_FREQUENCY.txt', 'C9_MASTER_PHASE.txt', 'C9_SLAVE_FREQUENCY.txt', 'C9_SLAVE_PHASE.txt', 'crosshead_pos.txt', 'strain.txt', 'stress.txt', 'slit_h.txt', 'slit_v.txt', 'cryo_temp1.txt', 'cryo_temp2.txt', 'maxheaterOP1.txt', 'maxheaterOP2.txt', 'TC_2.txt', 'TC_4.txt', 'TC_5.txt', 'TC_6.txt', 'FU

### Extracting times

In [ ]:
directory = "D:\\"
time = np.array([])
Experiment_index = np.array([])
for foldername in os.listdir(directory):
    if foldername[0:5] == 'cycle':
        n = FindDel(os.listdir("D:\\"+foldername))
        print(n)
        for filename in os.listdir("D:\\"+foldername):
            if filename[-4:] == ".txt":
                file_path = directory+foldername+"\\"+filename
                df = pd.read_csv(file_path, sep = "\t", names = ["Date\time", "other"])
                li = df["Date\time"].values
                if os.stat(file_path).st_size == 0: # remove empty files
                    pass
                elif len(str(df.iloc[0].values[0])) < 18: # remove the files without time in the first colmn
                    print(filename)
                    pass
                elif li[0][10] =="T":
                    for i in range(len(li)): # fix the T issue in some times 
                        untdate = li[0].replace('T',' ')
                        li = np.append(li,[untdate])
                        li = np.delete(li,[0], axis = None)
                    time = np.concatenate((a,li),axis=None)
                    Experiment_index = np.append(Experiment_index,[filename[:n-1]])
                else: # Normal time columns
                    time = np.concatenate((a,li),axis=None)
                    Experiment_index = np.append(Experiment_index,[filename[:n-1]])
                    #print(a)
print(len(time))
print(time)
print(Experiment_index)
print(len(Experiment_index))

### Making a dictionary from spreadsheet of filenames

In [4]:
path = "C:\\Users\\zahaab\\ENGIN_X_DATA_CLEANING\\File_names_2013-19.xlsx"
def Translator(path):
    xls = pd.ExcelFile(path)
    df = xls.parse(sheet_name = "Sheet1")
    df = df[["Filename", "Column_name"]]
    df = df.fillna(0)
    true_name_translater = dict(zip(df["Filename"],df["Column_name"]))
    s = set(true_name_translater.values())
    col_names = list(s)
    col_names.remove(0)
    col_names.append("measured_temperature_U")
    col_names.append("unregistered_value")
    return true_name_translater, col_names

true_name_translater, col_names = Translator(path)

### Making function(s) for extracting data from files 

In [92]:
def NewRun(run_number,run_data, time_list):
    """This activates if a new run has started, it makes a new list of dictionaries and makes a CSV out of current data"""
    df1 = pd.DataFrame(columns = ["Date/Time"])
    for i in time_list:
        row = np.array([i])
        column_names = np.array(["Date/Time"])
        for j in run_data:
            try:
                row = np.append(row, [j[1][i]])
                column_names = np.append(column_names, [j[0]])
            except:
                pass
        df2 = pd.DataFrame([row], columns = column_names)
        df1 = df1.append(df2, sort=True)
    df1.set_index("Date/Time", drop=True)
    df1.to_csv("C:\\Users\\zahaab\\test_runs\\"+run_number)
    
    run_data = []
    time_list = np.array([])
    for i,j in enumerate(col_names):
        run_data.append((j,{}))
    
    return run_data,time_list

def Find_dict(run_data, trans_filename):
    for i,j in enumerate(run_data):
        if j[0] == trans_filename:
            return i

def AddData(file_path, filename, n, run_data, time_list):
    """This takes data from a txt file and puts it into a list of times and the dictionaries"""
    df = pd.read_csv(file_path, sep = "\t", names = ["Date/Time", str(true_name_translater[filename[n:]])])
    file_times = df["Date/Time"].values
    file_data = df[str(true_name_translater[filename[n:]])].values 
    dict_index = Find_dict(run_data, true_name_translater[filename[n:]])
    if os.stat(file_path).st_size == 0: # remove empty files
        pass
    elif len(str(df.iloc[0].values[0])) < 18: # remove the files without time in the first colmn
        pass
    elif file_times[0][10] =="T":
        for i in range(len(li)): # fix the T issue in some times 
            untdate = file_times[0].replace('T',' ')
            file_times = np.append(file_times,[untdate])
            file_times = np.delete(file_times,0, axis = None)
        time_list = np.unique(np.append(time_list, file_times))
        for i, j in zip(file_times,file_data):
            if "temperature" in true_name_translater[filename[n:]]:#Alot of tempreture files, may be some overlap in time
                try:
                    a = run_data[dict_index][1][i]
                    run_data[-2][1][i] = j
                except:
                    run_data[dict_index][1][i] = j
            else:#rest of the files 
                try:
                    a = run_data[dict_index][1][i]
                    run_data[-1][1][i] = [run_data[dict_index][0],j]
                except:
                    run_data[dict_index][1][i] = j
    else: # Normal time columns
        time_list = np.unique(np.append(time_list, file_times))
        for i, j in zip(file_times,file_data):
            if "temperature" in true_name_translater[filename[n:]]:
                try:
                    a = run_data[dict_index][1][i]
                    run_data[-2][1][i] = j
                except:
                    run_data[dict_index][1][i] = j
            else:
                try:
                    a = run_data[dict_index][1][i]
                    run_data[-1][1][i] = [run_data[dict_index][0],j]
                except:
                    run_data[dict_index][1][i] = j
    return run_data, time_list
    
            
direc = "C:\\Users\\zahaab\\test_case2_no_raw"

for foldername in os.listdir(direc):
    folder_path = direc+"\\"+foldername
    n = FindDel(os.listdir(folder_path))
    filenames = os.listdir(folder_path)
    run_data = np.array([])
    time_list = np.array([])
    finale_file = 0
    for k, filename in enumerate(filenames):
        print(filename)
        file_path = folder_path+'\\'+filename
        if RunIndex(filenames[k],n) != RunIndex(filenames[k-1],n):
            run_data,time_list = NewRun(RunIndex(filenames[k-1],n), run_data, time_list)
            run_data,time_list = AddData(file_path, filename, n, run_data, time_list)
        else:
            run_data,time_list = AddData(file_path, filename, n, run_data, time_list)
        finale_file = filenames[k]
    NewRun(RunIndex(filenames[k],n), run_data, time_list)

ENG44295_Theta.txt
ENG44295_X.txt
ENG44295_Y.txt
ENG44295_Z.txt
ENG44296_Theta.txt
ENG44296_X.txt
ENG44296_Y.txt
ENG44296_Z.txt
ENG44297_Theta.txt
ENG44297_X.txt
ENG44297_Y.txt
ENG44297_Z.txt
ENG44298_Theta.txt
ENG44298_X.txt
ENG44298_Y.txt
ENG44298_Z.txt
ENG44299_Theta.txt
ENG44299_X.txt
ENG44299_Y.txt
ENG44299_Z.txt
ENG44300_Theta.txt
ENG44300_X.txt
ENG44300_Y.txt
ENG44300_Z.txt
ENG44301_Theta.txt
ENG44301_X.txt
ENG44301_Y.txt
ENG44301_Z.txt
ENG44302_Theta.txt
ENG44302_X.txt
ENG44302_Y.txt
ENG44302_Z.txt
ENG44303_Theta.txt
ENG44303_X.txt
ENG44303_Y.txt
ENG44303_Z.txt
ENG44304_Theta.txt
ENG44304_X.txt
ENG44304_Y.txt
ENG44304_Z.txt
ENG44305_Theta.txt
ENG44305_X.txt
ENG44305_Y.txt
ENG44305_Z.txt
ENG44306_Theta.txt
ENG44306_X.txt
ENG44306_Y.txt
ENG44306_Z.txt
ENG44307_Theta.txt
ENG44307_X.txt
ENG44307_Y.txt
ENG44307_Z.txt
ENG44308_Theta.txt
ENG44308_X.txt
ENG44308_Y.txt
ENG44308_Z.txt
ENG44309_Theta.txt
ENG44309_X.txt
ENG44309_Y.txt
ENG44309_Z.txt
ENG44310_Theta.txt
ENG44310_X.txt
ENG443

ENG44424_X.txt
ENG44424_Y.txt
ENG44424_Z.txt
ENG44425_Theta.txt
ENG44425_X.txt
ENG44425_Y.txt
ENG44425_Z.txt
ENG44426_Theta.txt
ENG44426_X.txt
ENG44426_Y.txt
ENG44426_Z.txt
ENG44427_Theta.txt
ENG44427_X.txt
ENG44427_Y.txt
ENG44427_Z.txt
ENG44428_Theta.txt
ENG44428_X.txt
ENG44428_Y.txt
ENG44428_Z.txt
ENG44429_Theta.txt
ENG44429_X.txt
ENG44429_Y.txt
ENG44429_Z.txt
ENG44430_Theta.txt
ENG44430_X.txt
ENG44430_Y.txt
ENG44430_Z.txt
ENG44431_Theta.txt
ENG44431_X.txt
ENG44431_Y.txt
ENG44431_Z.txt
ENG44432_Theta.txt
ENG44432_X.txt
ENG44432_Y.txt
ENG44432_Z.txt
ENG44433_Theta.txt
ENG44433_X.txt
ENG44433_Y.txt
ENG44433_Z.txt
ENG44434_Theta.txt
ENG44434_X.txt
ENG44434_Y.txt
ENG44434_Z.txt
ENG44435_Theta.txt
ENG44435_X.txt
ENG44435_Y.txt
ENG44435_Z.txt
ENG44436_Theta.txt
ENG44436_X.txt
ENG44436_Y.txt
ENG44436_Z.txt
ENG44437_Theta.txt
ENG44437_X.txt
ENG44437_Y.txt
ENG44437_Z.txt
ENG44438_Theta.txt
ENG44438_X.txt
ENG44438_Y.txt
ENG44438_Z.txt
ENG44439_Theta.txt
ENG44439_X.txt
ENG44439_Y.txt
ENG44439_Z

ENG44549_Y.txt
ENG44549_Z.txt
ENG44550_Slit_H.txt
ENG44550_Slit_V.txt
ENG44550_Slit_X.txt
ENG44550_Theta.txt
ENG44550_X.txt
ENG44550_Y.txt
ENG44550_Z.txt
ENG44551_Slit_H.txt
ENG44551_Slit_V.txt
ENG44551_Slit_X.txt
ENG44551_Theta.txt
ENG44551_X.txt
ENG44551_Y.txt
ENG44551_Z.txt
ENG44552_Slit_H.txt
ENG44552_Slit_V.txt
ENG44552_Slit_X.txt
ENG44552_Theta.txt
ENG44552_X.txt
ENG44552_Y.txt
ENG44552_Z.txt
ENG44553_Slit_H.txt
ENG44553_Slit_V.txt
ENG44553_Slit_X.txt
ENG44553_Theta.txt
ENG44553_X.txt
ENG44553_Y.txt
ENG44553_Z.txt
ENG44554_Slit_H.txt
ENG44554_Slit_V.txt
ENG44554_Slit_X.txt
ENG44554_Theta.txt
ENG44554_X.txt
ENG44554_Y.txt
ENG44554_Z.txt
ENG44555_Slit_H.txt
ENG44555_Slit_V.txt
ENG44555_Slit_X.txt
ENG44555_Theta.txt
ENG44555_X.txt
ENG44555_Y.txt
ENG44555_Z.txt
ENG44556_Slit_H.txt
ENG44556_Slit_V.txt
ENG44556_Slit_X.txt
ENG44556_Theta.txt
ENG44556_X.txt
ENG44556_Y.txt
ENG44556_Z.txt
ENG44557_Slit_H.txt
ENG44557_Slit_V.txt
ENG44557_Slit_X.txt
ENG44557_Theta.txt
ENG44557_X.txt
ENG44557

ENG44616_Slit_H.txt
ENG44616_Slit_V.txt
ENG44616_Slit_X.txt
ENG44616_Theta.txt
ENG44616_X.txt
ENG44616_Y.txt
ENG44616_Z.txt
ENG44641_dash_controls.txt
ENG44641_Theta.txt
ENG44641_X.txt
ENG44641_Y.txt
ENG44641_Z.txt
ENG44642_dash_controls.txt
ENG44642_MonCounts.txt
ENG44642_Theta.txt
ENG44642_X.txt
ENG44642_Y.txt
ENG44642_Z.txt
ENG44643_MonCounts.txt
ENG44644_MonCounts.txt
ENG44645_MonCounts.txt
ENG44646_MonCounts.txt
ENG44647_MonCounts.txt
ENG44648_MonCounts.txt
ENG44649_MonCounts.txt
ENG44650_MonCounts.txt
ENG44651_MonCounts.txt
ENG44652_MonCounts.txt
ENG44653_MonCounts.txt
ENG44654_MonCounts.txt
ENG44655_MonCounts.txt
ENG44656_MonCounts.txt
ENG44657_MonCounts.txt
ENG44658_MonCounts.txt
ENG44659_MonCounts.txt
ENG44660_MonCounts.txt
ENG44661_MonCounts.txt
ENG44662_MonCounts.txt
ENG44663_MonCounts.txt
ENG44664_MonCounts.txt
ENG44665_MonCounts.txt
ENG44666_MonCounts.txt
ENG44667_MonCounts.txt
ENG44668_MonCounts.txt
ENG44669_MonCounts.txt
ENG44670_MonCounts.txt
ENG44671_MonCounts.txt
ENG4

### Finding how to deal with Stress rig data 

In [11]:
direct = "D:\\stuff\\OtherData\\cycle_13_1\\Stress Rig"

for filename in os.listdir(direct):
    file_path = direct+"\\"+filename
    print(filename)
    df = pd.read_csv(file_path, sep = "|", skipinitialspace = True, skiprows = [0,1])
    print(df)
df

Stress Rig-2013 06 13, June Thursday-16,02,11.dat
        Date/Time         Position        Averaged Position (mm)    \
0  2013/06/13 16:02:11         20.276296                 20.275231   
1  2013/06/13 16:02:12         20.275092                 20.275199   
2  2013/06/13 16:02:13         20.274723                 20.275328   
3  2013/06/13 16:02:14         20.275373                 20.275243   
4  2013/06/13 16:02:15         20.274824                 20.275262   

   Load          Averaged Load (MPa)    Strain 1        Averaged Strain 1 (%)  
0      0.116931               0.036325        0.052738               0.053590  
1      0.076992               0.057372        0.052667               0.053333  
2      0.117387               0.069551        0.050459               0.052687  
3      0.118754               0.087501        0.053239               0.053000  
4      0.020197               0.090052        0.053792               0.052579  
Stress Rig-2013 06 13, June Thursday-18,36,30.dat

             Date/Time         Position        Averaged Position (mm)    \
0       2013/07/12 18:26:55          9.119666                  9.119846   
1       2013/07/12 18:26:56          9.118843                  9.119609   
2       2013/07/12 18:26:57          9.118897                  9.119374   
3       2013/07/12 18:26:58          9.120226                  9.119508   
4       2013/07/12 18:26:59          9.120453                  9.119617   
5       2013/07/12 18:27:00          9.119529                  9.119589   
6       2013/07/12 18:27:01          9.120107                  9.119842   
7       2013/07/12 18:27:02          9.119886                  9.120040   
8       2013/07/12 18:27:03          9.119588                  9.119913   
9       2013/07/12 18:27:04          9.120399                  9.119902   
10      2013/07/12 18:27:05          9.118682                  9.119733   
11      2013/07/12 18:27:06          9.117854                  9.119282   
12      2013/07/12 18:27:

,Date/Time,Position,Averaged Position (mm),Load,Averaged Load (MPa),Strain 1,Averaged Strain 1 (%)
0,2013/07/15 14:38:36,11.674732,11.674963,-0.318130,-0.321567,0.007048,0.007356
1,2013/07/15 14:38:37,11.675316,11.674941,-0.260695,-0.312952,0.006648,0.007188
2,2013/07/15 14:38:38,11.672592,11.674448,-0.435002,-0.356782,0.007064,0.007048
3,2013/07/15 14:38:39,11.674756,11.674440,-0.293532,-0.330865,0.007173,0.007054
4,2013/07/15 14:38:40,11.674589,11.674397,-0.478078,-0.357088,0.007288,0.007044
5,2013/07/15 14:38:41,11.673325,11.674116,-0.501147,-0.393691,0.007343,0.007103
6,2013/07/15 14:38:42,11.674273,11.673907,-0.469957,-0.435543,0.007599,0.007293
7,2013/07/15 14:38:43,11.673874,11.674164,-0.226917,-0.393926,0.006889,0.007258
8,2013/07/15 14:38:44,11.674190,11.674050,-0.240334,-0.383287,0.007788,0.007381
9,2013/07/15 14:38:45,11.672050,11.673542,-0.374624,-0.362596,0.007718,0.007467


In [84]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                    index=[0, 1, 2, 3]) 

df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']},
                    index=[4, 5, 6, 7])

df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                    'B': ['B8', 'B9', 'B10', 'B11'],
                    'C': ['C8', 'C9', 'C10', 'C11'],
                    'D': ['D8', 'D9', 'D10', 'D11']},
                    index=[8, 9, 10, 11])

df4 = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'],
                    'D': ['D2', 'D3', 'D6', 'D7'],
                    'F': ['F2', 'F3', 'F6', 'F7']},
                    index=[2, 3, 6, 7])

result = result = pd.concat([df1, df4], axis=1, sort=False)

result

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3
6,NaN,NaN,NaN,NaN,B6,D6,F6
7,NaN,NaN,NaN,NaN,B7,D7,F7


In [98]:
d = "C:\\Users\\zahaab\\test_runs"
df1 = pd.DataFrame()
for filename in os.listdir(d):
    #print(filename)
    file_path = d+"\\"+filename
    df2 = pd.read_csv(file_path,index_col=0)
    df2["Run_Number"] = filename
    df1 = df1.append(df2, ignore_index=False, sort=False)

print(len(df1["Date/Time"].values))
print(len(np.unique(df1["Date/Time"].values)))
      
direct = "D:\\stuff\\OtherData\\cycle_13_1\\Stress Rig"
df3 = pd.DataFrame()
for filename in os.listdir(direct):
    file_path = direct+"\\"+filename
    df4 = pd.read_csv(file_path, sep = "|", skipinitialspace = True, skiprows = [0,1])
    df3 = df3.append(df4, ignore_index=False, sort=False)
#print(df3)
df3 = df3.rename(index = str, columns = dict(zip(
    df3.columns.values.tolist(), ['rig_{0}'.format(i).replace(" ", "") for i in df3.columns.values.tolist()])))
df3 = df3.rename(index = str, columns = {'rig_Date/Time' : 'Date/Time'})

print(len(df3["Date/Time"].values))
print(len(np.unique(df3["Date/Time"].values)))

df1.set_index("Date/Time")
df3.set_index("Date/Time")

8result = pd.concat([df3, df1], axis=1, sort=False, join='outer')

print(df1)
print(df3)
result 

31359
31359
236731
232091


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [102]:
print(len(df3["Date/Time"].values))
print(len(np.unique(df3["Date/Time"].values)))

b = df3.drop_duplicates()
print(len(b["Date/Time"].values))
print(len(np.unique(b["Date/Time"].values)))

from collections import Counter
a = (Counter(b["Date/Time"].values) - Counter(set(b["Date/Time"].values))).keys()
print(a)

236731
232091
236608
232091
dict_keys(['2013/06/13 18:38:37  ', '2013/06/13 18:39:04  ', '2013/06/13 18:39:15  ', '2013/06/13 18:39:24  ', '2013/06/13 18:39:30  ', '2013/06/13 18:43:31  ', '2013/06/13 18:43:37  ', '2013/06/13 18:44:21  ', '2013/06/13 18:47:50  ', '2013/06/13 18:47:56  ', '2013/06/13 18:48:43  ', '2013/06/13 18:52:12  ', '2013/06/13 18:52:19  ', '2013/06/13 18:53:05  ', '2013/06/13 18:53:09  ', '2013/06/13 18:56:00  ', '2013/06/13 18:56:03  ', '2013/06/13 18:56:11  ', '2013/06/13 18:56:17  ', '2013/06/13 19:03:01  ', '2013/06/13 19:03:05  ', '2013/06/13 19:03:10  ', '2013/06/13 19:03:54  ', '2013/06/13 19:03:59  ', '2013/06/13 19:11:06  ', '2013/06/13 19:11:12  ', '2013/06/13 19:11:20  ', '2013/06/13 19:11:47  ', '2013/06/13 19:11:56  ', '2013/06/13 19:12:00  ', '2013/06/13 19:15:35  ', '2013/06/13 19:15:42  ', '2013/06/13 19:15:51  ', '2013/06/13 19:18:46  ', '2013/06/13 19:19:24  ', '2013/06/13 19:19:31  ', '2013/06/13 19:19:36  ', '2013/06/13 19:19:39  ', '2013/06/13

In [54]:
li = ["four","five","six"]
li2 = ['rig_{0}'.format(i) for i in li]
li2

['rigfour', 'rigfive', 'rigsix']